# Class 5: APIs

## APIs

- They are very powerful
- Can be used from any programming language
- Not expecting you to use them in your Final Project

## APIs, conceptually

![Diagram showing how online payments work: Expedia talks to Delta, Delta talks to Stripe, Stripe talks to Visa, and Visa talks to Chase](extras/img/apis_conceptually/payments.png)

![Diagram showing how notifications flow through systems](extras/img/apis_conceptually/notifications.png)

![Diagram showing relationship between human languages, programming languages, and APIs](extras/img/apis_conceptually/languages.png)

interactions between systems ↔️

## Ways to get data

Method | How it happens | Pros | Cons
--- | :--- | :--- | :---
**Bulk** | Download, someone hands you a flash drive, etc. | Fast, one-time transfer | Can be large
**Scraping** | Data only available through a web site, PDF, or doc | You can turn anything into data | Tedious; fragile
**APIs** | If organization makes one available | Usually allows some filtering; can always pull latest-and-greatest | Requires network connection for every call; higher barrier to entry (reading documentation); subject to availability and performance of API

## Scraping

Common tools:

- [Beautiful Soup package](https://realpython.com/beautiful-soup-web-scraper-python/)
- [pandas' `read_html()`](https://pandas.pydata.org/docs/user_guide/io.html#html)

_Please pray to the Demo Gods that these all work and there's no profanity_

Pull table from [Wikipedia's list of countries by area](https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area#Countries_and_dependencies_by_area):

In [1]:
import pandas as pd

tables = pd.read_html(
    "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area",
    match="Country / dependency",
)
countries = tables[0]
countries

,Unnamed: 0,Country / dependency,Totalin km2 (mi2),Landin km2 (mi2),Waterin km2 (mi2),%water,Unnamed: 6
0,NaN,Earth,"510,072,000 (196,940,000)","148,940,000 (57,506,000)","361,132,000 (139,434,000)",70.8,NaN
1,1,Russia,"17,098,246 (6,601,667)","16,378,410 (6,323,737)","719,836 (277,930)",4.2,[b]
2,NaN,Antarctica,"14,200,000 (5,480,000)","14,200,000 (5,480,000)",0,0.0,[c]
3,2,Canada,"9,984,670 (3,855,100)","9,093,507 (3,511,021)","891,163 (344,080)",8.9,[d]
4,3 or 4[e],China,"9,596,961 (3,705,406)","9,326,410 (3,600,950)","270,550 (104,460)",2.8,[f][1]
...,...,...,...,...,...,...,...
260,NaN,Ashmore and Cartier Islands (Australia),5.0 (1.9),5.0 (1.9),0,0.0,NaN
261,NaN,Spratly Islands (disputed),2.0 (0.77),2.0 (0.77),0,0.0,NaN
262,NaN,Coral Sea Islands (Australia),2.0 (0.77),2.0 (0.77),0,0.0,NaN
263,194,Monaco,2.0 (0.77),2.0 (0.77),0,0.0,[dp]


### Data is only available if it's available

## API calls in the wild

1. Go to [Candidates page on fec.gov](https://www.fec.gov/data/candidates/?has_raised_funds=true&is_active_candidate=true).
1. Right click and `Inspect`.
   - [More info about opening Developer Tools in various browsers.](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_are_browser_developer_tools#how_to_open_the_devtools_in_your_browser)
1. Go to the `Network` tab and reload.
1. Filter to `XHR`.
1. Click the API call.

We only see this because the tables on [fec.gov](https://fec.gov) are [rendered client-side](https://www.solutelabs.com/blog/client-side-vs-server-side-rendering-what-to-choose-when) using their JSON API. That won't be the case for all tables on all sites.

### Parts of a URL

![URL structure](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/Web_mechanics/What_is_a_URL/mdn-url-all.png)

[source](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL#basics_anatomy_of_a_url)

For APIs:

- Often split into "base URL" + "endpoint"
- Endpoints are like function names: they represent the information you are retrieving or thing you are trying to do
- Parameters are like function arguments:
   - They allow options to be specified
   - Some are required, some are optional
   - They will differ from one endpoint/function to another
- Anchors won't be used

### API documentation

[FEC API](https://api.open.fec.gov/developers/)

### Try it out

1. Visit https://www.fec.gov/data/candidates/
1. [Open Developer Tools](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_are_browser_developer_tools#how_to_open_the_devtools_in_your_browser).
1. Reload the page.
1. In the Network tab's request list, right-click the API call.
1. Click `Open in New Tab`.
1. Replace the `api_key` value with `DEMO_KEY`.

## API calls from Python

Usually one of two ways:

- A software development kit (SDK) like [sodapy](https://pypi.org/project/sodapy/)
   - Abstracts the details away
   - Not available for all APIs
   - May have limitations
- [The `requests` package](https://docs.python-requests.org/) (nothing to do with 311 requests)

In [2]:
import requests

params = {
    "api_key": "DEMO_KEY",
    "q": "Jimmy McMillan",
    "sort": "-first_file_date",
}
response = requests.get("https://api.open.fec.gov/v1/candidates/", params=params)
data = response.json()
data

{'api_version': '1.0',
 'results': [{'name': 'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH',
   'flags': 'P60016805',
   'office_full': 'President',
   'last_f2_date': '2015-10-13',
   'federal_funds_flag': False,
   'election_years': [2016],
   'incumbent_challenge_full': 'Open seat',
   'inactive_election_years': None,
   'candidate_status': 'N',
   'office': 'P',
   'cycles': [2016, 2018],
   'party': 'REP',
   'election_districts': ['00'],
   'load_date': '2018-02-17T09:16:20+00:00',
   'active_through': 2016,
   'candidate_id': 'P60016805',
   'district_number': 0,
   'district': '00',
   'last_file_date': '2015-10-13',
   'state': 'US',
   'party_full': 'REPUBLICAN PARTY',
   'incumbent_challenge': 'O',
   'candidate_inactive': False,
   'has_raised_funds': False,
   'first_file_date': '2015-10-13'},
  {'name': 'MCMILLAN, JIMMY (AKA) JAMES ',
   'flags': 'P60003290',
   'office_full': 'President',
   'last_f2_date': '2011-02-07',
   'federal_funds_flag': False,
   'election_years': [19

### Retrieving nested data

In [3]:
data["results"][0]["name"]

'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH'

### In-class exercise

[Geocode](https://en.wikipedia.org/wiki/Address_geocoding) an address from Python using the [Nominatim API](https://nominatim.org/release-docs/develop/api/Search/). Print out the latitude and longitude. Any address is fine:

- Your own
- This building
- etc.

### Reading into a DataFrame

In [4]:
params = {
    "api_key": "DEMO_KEY",
    "has_raised_funds": "true",
}
response = requests.get("https://api.open.fec.gov/v1/candidates/", params=params)
data = response.json()

pd.DataFrame(data["results"])

,election_years,has_raised_funds,cycles,party_full,office_full,party,candidate_status,candidate_id,first_file_date,candidate_inactive,...,last_file_date,load_date,district,incumbent_challenge,election_districts,federal_funds_flag,inactive_election_years,last_f2_date,active_through,office
0,[2022],True,"[2022, 2024]",REPUBLICAN PARTY,House,REP,P,H2CO07170,2021-12-27,False,...,2022-08-10,2023-03-09T10:16:03+00:00,07,O,[07],False,None,2022-08-10,2022,H
1,[2022],True,[2022],REPUBLICAN PARTY,House,REP,C,H2UT03280,2020-03-24,False,...,2022-03-21,2022-04-13T21:10:09+00:00,03,C,[03],False,None,2022-03-21,2022,H
2,"[2012, 2018]",True,"[2012, 2014, 2016, 2018, 2020]",CONSTITUTION PARTY,Senate,CON,P,S2UT00229,2012-02-08,False,...,2018-04-23,2019-03-27T16:02:41+00:00,00,O,"[00, 00]",False,None,2018-04-23,2018,S
3,[2020],True,[2020],REPUBLICAN PARTY,House,REP,C,H0TX22260,2019-10-17,False,...,2019-10-17,2020-03-18T21:13:37+00:00,22,O,[22],False,None,2019-10-17,2020,H
4,"[1976, 1978]",True,"[1976, 1978, 1980]",REPUBLICAN PARTY,House,REP,P,H6PA16106,1976-04-12,False,...,1978-07-05,2002-03-30T00:00:00+00:00,16,None,"[16, 16]",False,None,1978-07-05,1978,H
5,[2012],True,"[2012, 2014, 2016]",REPUBLICAN PARTY,House,REP,P,H2CA01110,2012-02-22,False,...,2012-02-22,2013-04-26T09:04:30+00:00,01,C,[01],False,None,2012-02-22,2012,H
6,[2018],True,[2018],DEMOCRATIC PARTY,House,DEM,C,H8CO06237,2017-04-26,False,...,2017-04-26,2017-08-01T20:57:28+00:00,06,C,[06],False,None,2017-04-26,2018,H
7,[2008],True,"[2006, 2008, 2010, 2012, 2014, 2016]",DEMOCRATIC PARTY,President,DEM,N,P80002926,2005-10-12,False,...,2007-03-13,2016-11-17T06:10:48+00:00,00,O,[00],False,None,2007-03-13,2008,P
8,"[2022, 2024]",True,"[2022, 2024]",DEMOCRATIC PARTY,House,DEM,N,H2CA30291,2021-01-16,False,...,2022-07-15,2023-01-12T22:24:01+00:00,32,C,"[32, 32]",False,None,2022-07-15,2024,H
9,[2022],True,"[2022, 2024]",DEMOCRATIC-FARMER-LABOR,House,DFL,P,H2MN07162,2022-06-06,False,...,2022-06-06,2023-03-09T10:16:03+00:00,07,C,[07],False,None,2022-06-06,2022,H


## Back to 311 data

From [NYC Open Data Portal dataset page](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/data), click `Export` -> `SODA API` -> `API Docs`.

### Most open data sites have APIs

Often built on platforms that provide them, e.g.

- [NYC Open Data Portal](https://opendata.cityofnewyork.us/) built on [Socrata](https://dev.socrata.com/)
- [data.gov built on CKAN](https://www.data.gov/developers/apis)

### Example: 311 requests from the last week

In [5]:
from datetime import datetime, timedelta

now = datetime.utcnow()
now

datetime.datetime(2023, 7, 26, 5, 27, 36, 682946)

In [6]:
start = now - timedelta(weeks=1)
start

datetime.datetime(2023, 7, 19, 5, 27, 36, 682946)

In [7]:
start.isoformat()

'2023-07-19T05:27:36.682946'

Using the [Socrata query language (SoQL)](https://dev.socrata.com/docs/queries/):

In [8]:
data_id = "erm2-nwe9"
params = {
    "$where": f"created_date between '{start.isoformat()}' and '{now.isoformat()}'",
}

url = f"https://data.cityofnewyork.us/resource/{data_id}.json"
response = requests.get(url, params=params)
data = response.json()

data

[{'unique_key': '58299170',
  'created_date': '2023-07-25T12:00:00.000',
  'agency': 'DSNY',
  'agency_name': 'Department of Sanitation',
  'complaint_type': 'Derelict Vehicles',
  'descriptor': 'Derelict Vehicles',
  'location_type': 'Street',
  'incident_zip': '11208',
  'incident_address': '851 LIBERTY AVENUE',
  'street_name': 'LIBERTY AVENUE',
  'cross_street_1': 'MONTAUK AVENUE',
  'cross_street_2': 'MILFORD STREET',
  'address_type': 'ADDRESS',
  'city': 'BROOKLYN',
  'facility_type': 'DSNY Garage',
  'status': 'Open',
  'resolution_description': 'If the abandoned vehicle meets the criteria to be classified as a derelict (i.e. junk) the Department of Sanitation (DSNY) will investigate and tag the vehicle within three business days.',
  'resolution_action_updated_date': '2023-07-25T12:00:00.000',
  'community_board': '05 BROOKLYN',
  'bbl': '3039760070',
  'borough': 'BROOKLYN',
  'x_coordinate_state_plane': '1018044',
  'y_coordinate_state_plane': '185990',
  'open_data_channel_

Like the FEC, Socrata uses their own API to populate the tables when browsing data on sites powered by them.

**At-home exercise:** Try filtering a table on the [NYC Open Data Portal](https://data.cityofnewyork.us/), and find the API calls that makes.

In [9]:
pd.DataFrame(data)

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,...,location,intersection_street_1,intersection_street_2,closed_date,landmark,bridge_highway_name,bridge_highway_direction,bridge_highway_segment,taxi_pick_up_location,road_ramp
0,58299170,2023-07-25T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11208,851 LIBERTY AVENUE,LIBERTY AVENUE,...,"{'latitude': '40.677111838873174', 'longitude'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,58300487,2023-07-25T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,10452,54 FEATHERBED LANE,FEATHERBED LANE,...,"{'latitude': '40.84566751269749', 'longitude':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58300479,2023-07-25T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11355,36-12 COLLEGE POINT BOULEVARD,COLLEGE POINT BOULEVARD,...,"{'latitude': '40.76110025661531', 'longitude':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,58296645,2023-07-25T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,NaN,NaN,NaN,...,NaN,MIDWOOD STREET,MIDWOOD STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,58301747,2023-07-25T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,10305,141 MCCLEAN AVENUE,MCCLEAN AVENUE,...,"{'latitude': '40.597931393654854', 'longitude'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,58295908,2023-07-24T21:54:22.000,NYPD,New York City Police Department,Noise - Park,Loud Music/Party,Park/Playground,11212,POWELL PLAYGROUND,POWELL PLAYGROUND,...,"{'latitude': '40.671315301456914', 'longitude'...",GLENMORE AVENUE,PITKIN AVENUE,2023-07-24T22:49:47.000,POWELL PLAYGROUND,NaN,NaN,NaN,NaN,NaN
996,58301130,2023-07-24T21:54:15.000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10463,3015 KINGSBRIDGE TERRACE,KINGSBRIDGE TERRACE,...,"{'latitude': '40.87602365179769', 'longitude':...",WEST 230 STREET,WEST 231 STREET,2023-07-24T22:27:47.000,KINGSBRIDGE TERRACE,NaN,NaN,NaN,NaN,NaN
997,58303164,2023-07-24T21:54:09.000,DSNY,Department of Sanitation,Illegal Dumping,Removal Request,Street,11417,103-30 98 STREET,98 STREET,...,"{'latitude': '40.68265925235989', 'longitude':...",103 AVENUE,LIBERTY AVENUE,NaN,98 STREET,NaN,NaN,NaN,NaN,NaN
998,58299727,2023-07-24T21:53:41.000,NYPD,New York City Police Department,Illegal Fireworks,N/A,Street/Sidewalk,11228,1164 BAY RIDGE PARKWAY,BAY RIDGE PARKWAY,...,"{'latitude': '40.62215120216889', 'longitude':...",11 AVENUE,12 AVENUE,2023-07-24T22:38:39.000,BAY RIDGE PARKWAY,NaN,NaN,NaN,NaN,NaN


Coincidence there were exactly 1,000 results?

### Pagination

- Most APIs limit the number of results returned.
- [Socrata defaults to 1,000.](https://dev.socrata.com/docs/queries/limit.html)
- Need to use a loop with parameters like [`$limit`](https://dev.socrata.com/docs/queries/limit.html)+[`$offset`](https://dev.socrata.com/docs/queries/offset.html) (Socrata) or `page`+`per_page` ([FEC](https://api.open.fec.gov/developers/))
   - [`concat()`](https://pandas.pydata.org/pandas-docs/version/1.5/user_guide/merging.html#concatenating-objects) to DataFrame

## Things are going to differ by API

- Endpoints
- Supported parameters
- Response structure
   - [`json_normalize()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) can help
- Quality of documentation
- Helpfulness of errors
- Size/helpfulness of community

Gotta read and experiment.

## [Final Project](https://python-public-policy.afeld.me/en/{{school_slug}}/final_project.html)

- You should have received feedback on your proposal.
- Reminder that it's peer-graded.
   - You should see the notebooks you need to review come through {{lms_name}}.
   - This is an opportunity to see how different people solve different problems.
   - You will lose points if you don't complete your peer grading.

All together, let's make sure our notebooks are properly shared for peer grading:

1. Create a notebook for your Final Project, if you haven't already.
1. [Open the Sharing settings.](https://research.google.com/colaboratory/faq.html#notebook-storage)
1. Under `General access`, change to `LionMail`, then `Commenter`.

## [Time Out](https://bulletin.columbia.edu/sipa/teaching-guide/#supportservicestext)

You should have gotten an email; please take a moment now to RSVP.

## [Schedule](https://python-public-policy.afeld.me/en/{{school_slug}}/syllabus.html#schedule)

## [Course evaluations](https://coursefeedback.nyu.edu/nyu/)

They are:

- Totally anonymous
- Not visible to me until grades are released
- A big help. Some things I took from the past:
    - Making assignments more rigorous
    - Students are hungry for more
    - People like the in-class exercises

Please complete now, if you haven't already, as they are due tonight.